In [1]:
import cv2 as cv
import numpy as np

In [2]:
img = cv.imread("test.jpg")
height, width, _ = img.shape

In [3]:
colors = np.random.randint(0, 255, (80, 3))

In [4]:
black_image = np.zeros((height, width, 3), np.uint8)
black_image[:] = (100, 100, 0)

In [5]:
model = cv.dnn.readNetFromTensorflow("dnn/frozen_inference_graph_coco.pb",
	                                 "dnn/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt")

In [6]:
blob = cv.dnn.blobFromImage(img, swapRB=True)
model.setInput(blob)

In [7]:
boxes, masks = model.forward(["detection_out_final", "detection_masks"])
detection_count = boxes.shape[2]

In [8]:
for i in range(detection_count):
	box = boxes[0, 0, i]
	class_id = box[1]
	score = box[2]
	if score < 0.5:
		continue

	# Get box Coordinates
	x = int(box[3] * width)
	y = int(box[4] * height)
	x2 = int(box[5] * width)
	y2 = int(box[6] * height)

	cv.rectangle(img, (x, y), (x2, y2), (150, 100, 214), 1)

	roi = black_image[y:y2, x:x2]
	roi_height, roi_width, _ = roi.shape

	# Get the mask
	mask = masks[i, int(class_id)]
	mask = cv.resize(mask, (roi_width, roi_height))
	_, mask = cv.threshold(mask, 0.5, 255, cv.THRESH_BINARY)


	# Get mask coordinates
	contours, _ = cv.findContours(np.array(mask, np.uint8), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
	color = colors[int(class_id)]
	for cnt in contours:
		cv.fillPoly(roi, [cnt], (int(color[0]), int(color[1]), int(color[2])))

In [9]:
cv.imshow("Image", img)
cv.imshow("Black image", black_image)
cv.waitKey(0)

113